In [1]:
import re
import time
import requests
import pandas as pd
from datetime import datetime
from requests_html import HTML

In [2]:
tag = input('Enter tag you want to scrap --> ')
query_filter = input('Enter Query Filter(Newest, Active, Votes) --> ')
max_page = int(input('Enter Maximum Pages you want to scrap --> '))
page_size = int(input('Enter maximum page size(15, 30, 50) --> '))

Enter tag you want to scrap --> ios
Enter Query Filter(Newest, Active, Votes) --> Votes
Enter Maximum Pages you want to scrap --> 30
Enter maximum page size(15, 30, 50) --> 50


In [3]:
def clean_data(text, keyname):
    if keyname == 'votes':
        return text.replace('\nvotes', '')
    if keyname == 'tags':
        return text.replace(' ', ', ')
    else:
        return text

In [4]:
def parse_tagged_page(html):
    question_summuries = html.find('.question-summary')
    
    key_names = ['question', 'votes', 'tags']
    classes_needed = ['.question-hyperlink', '.vote', '.tags']
    
    datas = []  
    for q_el in question_summuries:
        ques_data = {}
        href = q_el.find('a', first=True)
        ID = re.findall('\d+', str(href))
        ques_data['id'] = ID[0]
        for i, _class in enumerate(classes_needed):
            sub_el = q_el.find(_class, first=True)
            ques_data[key_names[i]] = clean_data(sub_el.text, key_names[i])
        datas.append(ques_data)
    
    return datas

In [5]:
def extract_data_url(url):
    r = requests.get(url)
    if r.status_code not in range (200, 299):
        return []
    else:
        html_str = r.text
        html = HTML(html=html_str)
        return parse_tagged_page(html)

In [6]:
def get_url(tag, query_filter, max_page, page_size):
    base_url = 'https://stackoverflow.com/questions/tagged/'
    
    datas = []
    for p in range(max_page):
        page_num = str(p + 1)
        url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={page_size}"
        datas += extract_data_url(url)
        time.sleep(1.2)
        
    return datas

In [7]:
datas = get_url(tag, query_filter, max_page, page_size)
datas[0]

{'id': '1054558',
 'question': 'Vertically align text to top within a UILabel',
 'votes': '2259',
 'tags': 'ios, cocoa-touch, uikit, uilabel, text-alignment'}

In [8]:
df = pd.DataFrame(datas)
df.head()

,id,question,votes,tags
0,1054558,Vertically align text to top within a UILabel,2259,"ios, cocoa-touch, uikit, uilabel, text-alignment"
1,588866,What's the difference between the atomic and n...,1891,"ios, objective-c, properties, atomic, nonatomic"
2,1126726,How can I make a UITextField move up when the ...,1737,"ios, objective-c, uiscrollview, uitextfield, u..."
3,18746929,Using Auto Layout in UITableView for dynamic c...,1559,"ios, uitableview, autolayout, row-height"
4,31254725,Transport security has blocked a cleartext HTTP,1522,"ios, xcode, ios9, ios10, app-transport-security"


In [9]:
df.to_csv('stack_overflow.csv', index=False)